In [1]:
%load_ext autoreload
%autoreload 2
import santas_helpers

In [2]:
#!conda install --yes -c conda-forge python-igraph
import igraph

In [23]:
example_1, example_2, full = santas_helpers.get_input(day=7)
example_1

['light red bags contain 1 bright white bag, 2 muted yellow bags.',
 'dark orange bags contain 3 bright white bags, 4 muted yellow bags.',
 'bright white bags contain 1 shiny gold bag.',
 'muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.',
 'shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.',
 'dark olive bags contain 3 faded blue bags, 4 dotted black bags.',
 'vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.',
 'faded blue bags contain no other bags.',
 'dotted black bags contain no other bags.']

In [30]:
import re
def parse_rule(rule):
    bag, _, contents = map(str.strip, rule.partition('contain'))
    bag = bag.replace('bags', '').strip()
    rule_matcher = re.compile('(\d+) ([a-z ]+) bag')
    if contents == 'no other bags.':
        return (bag, [(0, 'no other bags')])
    else:
        rules = [rule_matcher.match(desc.strip()).groups()[0:2] for desc in contents.split(',')]
        rules = [(int(rule[0]), rule[1]) for rule in rules]
        return (bag, rules)
print(parse_rule(example_1[0]))

('light red', [(1, 'bright white'), (2, 'muted yellow')])


In [32]:
def build_graph(raw_rule_set):
    bag_graph = igraph.Graph(directed=True)
    rules = [parse_rule(raw_rule) for raw_rule in raw_rule_set]

    for bag, _ in rules:
        bag_graph.add_vertex(name=bag)
    bag_graph.add_vertex(name='no other bags')
        
    for bag, contents in rules:
        for count, color in contents:
            edge = bag_graph.add_edge(bag, color)
            edge['count'] = count
            
    return bag_graph
graph = build_graph(example_1)

In [33]:
def find_outer_bags(raw_rule_set, target_color):
    graph = build_graph(raw_rule_set)
    # use the inbound into shiny-gold to find all paths that can get to shiny gold,
    # instead of check each node to see if it has a path to shiny-gold
    all_paths = graph.get_all_simple_paths(target_color, mode=igraph.IN)
    
    outer_bags = set()
    for path in all_paths:
        for vert_id in path:
            outer_bags.add(graph.vs[vert_id]['name'])
    return outer_bags - {target_color}

find_outer_bags(example_1, 'shiny gold')

{'bright white', 'dark orange', 'light red', 'muted yellow'}

In [65]:
def count_bags_needed(raw_rule_set, target):
    def count_bags(graph, source):
        count = 1
        for edge in source.out_edges():        
            count += edge['count'] * count_bags(graph, edge.target_vertex)
        return count
    graph = build_graph(raw_rule_set)
    return count_bags(graph, graph.vs.find(name=target)) - 1
count_bags_needed(example_1, 'shiny gold')

32

In [66]:
count_bags_needed(example_2, 'shiny gold')

126

In [67]:
count_bags_needed(full, 'shiny gold')

8015